## Naturkartan
* this [Notebook](https://github.com/salgo60/svenska-vandringsleder/blob/main/Notebook/Naturkartan.ipynb)   

API
* [API](https://api.naturkartan.se/v2/sites/search.geojson)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-09-20 17:45:13.042474


## API

In [4]:
urlNaturkartan = "https://api.naturkartan.se/v2/sites/search.geojson"
header ="accept: text/plain;charset=utf-8"
import urllib3, json
import pandas as pd 

http = urllib3.PoolManager()
r = http.request(
    'GET',
    urlNaturkartan,
    headers={
        "accept": "application/json"
    })

print(r.status)
data = json.loads(r.data.decode('utf-8')) 
print(type(data))
#print(r.info()) 

200
<class 'dict'>


In [16]:

from pandas.io.json import json_normalize
df = json_normalize(data["features"])

<ipython-input-16-83164599b5c2>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(data["features"])


In [17]:
df

,id,type,geometry.type,geometry.coordinates,properties.url,properties.link,properties.name,properties.slug,properties.tags,properties.time,...,properties.wheelchair_tested,properties.business_site_data,properties.show_in_brand_site,properties.trail_status_reported_at,properties.business_site_data.address,properties.business_site_data.telephone,properties.business_site_data.website_url,properties.business_site_data.opening_hours,properties.color,properties.parent
0,18546,Feature,Point,"[16.4748505691457, 66.3160608416262]",https://www.naturkartan.se/sv/norrbottenslan/l...,None,"Dass Kattugglekojan, Laisdalen",laisdalen-fur-dass-vid-kattugglekojan,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18547,Feature,Point,"[16.4894806445951, 66.2808717169018]",https://www.naturkartan.se/sv/norrbottenslan/l...,None,"Dass Bárasjuhka, Laisdalen",laisdalen-fur-dass-barrastugan,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18523,Feature,Point,"[16.9704617765231, 66.2410323852801]",https://www.naturkartan.se/sv/norrbottenslan/l...,None,"Informationsskylt Hällbacken, Laisdalen",laisdalen-fur-informationstavla-hallbacken,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16724,Feature,Point,"[17.3718579395161, 60.1902146008368]",https://www.naturkartan.se/sv/upplandsstiftels...,None,"Utsiktsplats, Iggelbo",utsiktsplats-iggelbo,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8631,Feature,Point,"[17.4701607227325, 59.7352970701782]",https://www.naturkartan.se/sv/uppsalalan/rastp...,None,"Rastplats, Wikparken",rastplats-wikparken-3,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,12953,Feature,Point,"[18.6791918052287, 68.3102143133751]",https://www.naturkartan.se/sv/norrbottenslan/t...,None,Dass Abiskojåkka,tc-torrdass-4,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
727,1879,Feature,Point,"[18.001083, 59.439732]",https://www.naturkartan.se/sv/taby/hundbad-2,http://www.taby.se/hunditaby,Hundbadet vid Rösjön,hundbad-2,[],NaN,...,False,NaN,True,NaN,,NaN,None,[],NaN,NaN
728,6198,Feature,Point,"[18.0608364179077, 59.2123892104539]",https://www.naturkartan.se/sv/huddinge/agestab...,None,Fornborgen vid Ågesta,agestaborgen,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
729,16055,Feature,Point,"[11.9337059341437, 57.5045442752106]",https://www.naturkartan.se/sv/halland/parkerin...,None,Parkering Särö Västerskog,parkering-saro-vasterskog,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.to_csv("Baturkartan.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 57 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   id                                           731 non-null    object 
 1   type                                         731 non-null    object 
 2   geometry.type                                731 non-null    object 
 3   geometry.coordinates                         731 non-null    object 
 4   properties.url                               649 non-null    object 
 5   properties.link                              199 non-null    object 
 6   properties.name                              650 non-null    object 
 7   properties.slug                              650 non-null    object 
 8   properties.tags                              650 non-null    object 
 9   properties.time                              0 non-null      float64
 10  pr

## Wikidata